## gensim

- Gensim is an open-source library for unsupervised topic modeling and NLP, using modern statistical machine learning.
- Gensim is designed to handle large text collections using data streaming and incremental online algorithms, which differentiates it from most other machine learning software packages that target only in-memory processing.

In [ ]:
pip install gensim

In [ ]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."
doc_f = "Big data is a term used to refer to data sets that are too large or complex for traditional data-processing application software to adequately deal with."
doc_g = "Data with many cases offer greater statistical power, while data with higher complexity may lead to a higher false discovery rate"
doc_h = "Big data was originally associated with three key concepts: volume, variety, and velocity."
doc_i = "A 2016 definition states that 'Big data represents the information assets characterized by such a high volume, velocity and variety to require specific technology and analytical methods for its transformation into value'."
doc_j = "Data must be processed with advanced tools to reveal meaningful information."

In [ ]:
import gensim
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  RegexpTokenizer

tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')
s = PorterStemmer()

In [ ]:
mylist = [doc_a, doc_b, doc_c, doc_d, doc_e, doc_f, doc_g, \
          doc_h, doc_i, doc_j]

myword = []
for doc in mylist:
    mylower = doc.lower()
    mytoken = tokenizer.tokenize(mylower)
    mystopword = [i for i in mytoken if i not in stop_words]
    mystemmer = [s.stem(i) for i in mystopword ]
    myword.append(mystemmer)

In [ ]:
len(myword)

In [ ]:
mydic = corpora.Dictionary(myword)
myresult = [mydic.doc2bow(word) for word in myword]
myresult

In [ ]:
NUM_TOPICS = 7 #3개의 토픽, k=3
ldamodel = models.ldamodel.LdaModel(myresult, num_topics = NUM_TOPICS, id2word=mydic, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

## 그렇다면 Topic의 갯수를 어떻게 정할 것인가

- 대량의 문서에서 가장 적절한 Topic의 갯수를 넣어야 Model의 결과를 신뢰할 수 있을 것이다.
- 사전에 가장 적절한 Topic 갯수를 알고 있거나 또는 사전에 미리 분류해야 할 갯수를 알고 있으며 그 갯수를 지정하면 된다.
- 하지만, 대부분의 연구에서는 이러한 사전정보를 갖고 있지 않다.

### LDA Topic 갯수 지정

- 통계 모델링의 Topic 갯수를 지정하는 통계적 방법은 perplexity 또는 topic coherence 점수를 사용하는 것이다.
- Perplexity는 사전적 의미로는 '당혹', '혼란', '곤혹'이라는 뜻인데, 정보학에서는 혼란도를 의미한다.
- 이 수치는 특정 확률모델이 실제로 관측되는 값을 얼마나 잘 예측하는지 평가할 때 사용한다.
- Topic Modeling도 문헌 집합 내 용어 출현 횟수를 바탕으로, 문헌 내 주제 출현 확률과 주제 내 용어 출현 확률을 계산하는 확률 모델이므로 이것을 사용할 수 있다.
- Topic 갯수를 늘릴수록 Perplexity는 감소하는 경향을 보인다.
- 그러다가 특정 Topic 갯수 지점을 지나면 더 이상 감소하지 않고 수렴하는 지점이 등장하는데, 이 때의 perplexity가 해당 모델의 최종 perplexity로 본다.
- 결국 이 perplexity가 작으면 작을수록 해당 topic model은 실제 문헌 결과를 잘 반영한다는 뜻으로 학습이 잘 되었다고 평가할 수 있다.

- Topic Coherence는 실제로 사람이 해석하기에 적합한 평가 척도를 만들기 위해서 제시된 척도이다.
- 우선 topic modeling 결과로 나온 주제들에 대해 각각의 주제에서 상위 N개의 단어를 뽑는다.
- 모델링이 잘 될 수록 한 주제 안에는 의미론적으로 유사한 단어가 많이 모이게 마련이다.
- 따라서 상위 단어 간의 유사도를 계산하여 평균을 구하면 실제로 해당 주제가 의미론적으로 일치하는 단어들끼리 모여있는지 알 수 있다.

In [ ]:
from gensim.models import CoherenceModel

mycorherence = CoherenceModel(model = ldamodel, \
                         texts = myword, 
                             dictionary = mydic, topn = 5)

In [ ]:
print('\nPerplexity 지수 :', ldamodel.log_perplexity(myresult))
print('\nCoherence 지수 :', mycorherence.get_coherence())